

[Specifying multiple values as a derived table in a FROM clause](https://learn.microsoft.com/en-us/sql/t-sql/queries/table-value-constructor-transact-sql?view=sql-server-ver16#c-specifying-multiple-values-as-a-derived-table-in-a-from-clause)

In [ ]:
from utils import logs

In [ ]:
from sqlalchemy import create_engine, values, column, String, Integer, select, literal

In [ ]:
engine = create_engine('sqlite:///')

In [ ]:
example = values(
    column('id', Integer),
    column('name', String),
    name='example'
).data([
     (123, 'jack'),
     (456, 'john')
])
print(example)

with engine.begin() as con, logs():
    for row in con.execute(select(example)):
        print(row)

In [ ]:
x = select(literal(1), literal(2)).union(select(literal(3), literal(4)))
with engine.begin() as con, logs():
    for row in con.execute(x).mappings():
        print(row)

In [ ]:
import functools
import operator

def data_as_table(rows, *columns):
    result = values(
        column('id', Integer),
        column('name', String),
    ).data([
         tuple(r[c.key] for c in columns) for r in rows
    ])
    return select(result)

def _row(columns, row):
    return select(*(literal(row[c.key]).label(c.key) for c in columns))
    
def data_as_union(rows, *columns):
    # admittedly, a cryptic function.
    f = functools.partial(_row, columns)
    return functools.reduce(lambda a,b: a.union(b), map(f, rows))
    
def data_as_union_unrolled(rows, *columns):
    # TODO: testing
    buffer = []
    for row in rows:
        cells: list = []
        for col in columns:
            cells.append(literal(row[col.key]).label(col.key))
        buffer.append(select(*cells))
    # now chain all those select statments using 'union'
    result = buffer[0]
    for i in range(1, len(buffer)):
        result = result.union(buffer[i])
    return result

def data_as_select(rows, *columns, dialect):
    if dialect.name in ('mssql', 'snowflake'):
        return data_as_table(rows, *columns)
    return data_as_union(rows, *columns)

In [ ]:
x = data_as_select([{'id':1, 'name': 'jack'}, {'id':2, 'name':'john'}],     column('id', Integer),
    column('name', String), dialect=con.dialect)

In [ ]:
print(x)

In [ ]:
with engine.begin() as con, logs():
    for row in con.execute(select(x.cte())).mappings():
        print(row)